In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

### Optimization Problems

In [17]:
def solve_master():

    # model
    master = gp.Model()

    # variables + objective function
    z = master.addVars(i, q, obj = f, vtype = GRB.BINARY, name = "z")
    eta = master.addVar(1, obj = 1)

    # model sense
    master.ModelSense = GRB.MINIMIZE

    # constraints (do I need Benders constraints at this point)
    master.addConstr(gp.quicksum(gp.quicksum(b[i, q] * z[i, q] for i in H) for q in Q) >= gp.quicksum(W[k] for k in K))
    master.addConstrs(gp.quicksum(z[i, q]) <= 1 for i in H)

    # update and solve the model
    master.update()
    master.optimize()

    # optimal solution
    for var in master.getVars():
        print(f"{var.VarName} = {var.x}")

    return None

In [16]:
def solve_dual_subproblem():

    # model
    dual_subproblem = gp.Model()

    # variables + objective function
    alpha = dual_subproblem.addVars(k, obj = 1, name = "alpha")
    u = dual_subproblem.addVars(i, k, obj = z_hat, lb = 0, name = "u")
    v = dual_subproblem.addVars(i, obj = b * z_hat, lb = 0, name = "v")

    # model sense
    dual_subproblem.ModelSense = GRB.MAXIMIZE

    # constraints
    dual_subproblem.addConstrs(alpha[k] - u[a1][k] - u[a2][k] - W[k] * v[a1] <= F[a][k] for k in K for a in A_k_2)
    dual_subproblem.addConstrs(alpha[k] - u[a1][k] - W[k] * v[a1] <= F[a][k] for k in K for a in A_k_1)

    # update and solve the model
    dual_subproblem.update()
    dual_subproblem.optimize()

    # optimal solution
    for var in dual_subproblem.getVars():
        print(f"{var.VarName} = {var.x}")


    return None

In [ ]:
def solve_gap():

    # model
    gap = gp.Model()

    # variables + objective function
    x_hat = gap.addVars(a_i[k], k, obj = F, vtype = GRB.BINARY, name = "x_hat")

    # model sense
    gap.ModelSense = GRB.MINIMIZE

    # constraints
    gap.addConstrs(x[a_i[k]][k] = 1 for k in K)
    gap.addConstrs(gp.quicksum(W[k] * x[a_i[k]][k] for k in K) <= b[i][q_hat] for i in H(S'))

    # update and solve the model
    gap.update()
    gap.optimize()

    # optimal solution
    for var in gap.getVars():
        print(f"{var.VarName} = {var.x}")


### Algorithms

In [9]:
def benders_decomp():
    upper_bound = 0
    t = 0
    dual_extreme_pts = set()
    terminate = False

    while terminate == False:
        phi = solver_master() # solve the master problem and obtain the optimal objective function value

        if phi == upper_bound:
            terminate = True
        else:
            dual_soln = solve_dual_subproblem()
            dual_extreme_pts.add(dual_soln)

            if opt_dual_value + ... < upper_bound:
                upper_bound = opt_dual_value + ...

        t += 1

    return None

In [12]:
def exact_nmchlp():
    # Solve a problem
    # Solve a problem


    upper_bound = min(min, phi, np.inf)

    for i, q in zip(H(S*), Q_i(S*)):
        z_iq = 0
        # Solve problem

        if phi(MP^T) > upper_bound:
            z_iq = 1
        

    for i, q in zip(H*, Q_i*):
        z_iq = 1
        # Solve MP^T

        if phi(MP^T) > upper_bound:
            Q_i* = Q_i*.pop(q)

            if len(Q_i*) == 0:
                H* = H*.pop(i)

    S = set((i, q) for i, q in zip(H*, Q_i*))

    for S' subset S s.t. (19), (20) hold:
        # Sovle GAP problem

        if phi(GAP(z(S'))) < UB:
            UB = phi(GAP(z(S')))
                           
    return None